#단어 유사도 검사 메소드

In [1]:
from difflib import SequenceMatcher

In [2]:
def similarity(a, b):  
  return SequenceMatcher(None, a, b).ratio()

In [3]:
print(similarity('애월 한담해변','애월 한담해변'))

1.0


#코스데이터 불러오기

In [4]:
dir_path = "/content/drive/MyDrive/추천시스템 입문반/data/ml-latest-small"
course_path = dir_path + "/jeju_course_update.p"

In [5]:
import pandas as pd

In [6]:
course = pd.read_pickle(course_path)
course.head(2)

,Country Name,Country Initials,City,Package_Id,Package Title,Duration,Departure City,Departure Date,Places to Visit,Price,list path,list path(no airport),serial path(no airport)
0,대한민국,KOR,제주도,DCKC2TKO4,[청주출발]연합패키지 제주를 반하다 3일_관광호텔 화순곶자왈+서커스+석예원한방족욕+...,2박 3일,청주,2018.8.23,"수목원테마파크,카멜리아힐,성읍민속마을,아쿠아플라넷","306,000","[수목원테마파크, 카멜리아힐, 성읍민속마을, 아쿠아플라넷]","[수목원테마파크, 카멜리아힐, 성읍민속마을, 아쿠아플라넷]","수목원테마파크,카멜리아힐,성읍민속마을,아쿠아플라넷"
1,대한민국,KOR,제주도,DCKC2TKO3,[청주출발]연합패키지 노팁/노옵션 제주 3일_관광호텔 산방산유람선+ 화순곶자왈+서커...,2박 3일,청주,2018.8.23,"애월한담해안도로,용두암,카멜리아힐,수목원테마파크,산방산유람선,성읍민속마을,아쿠아플라넷","376,000","[애월한담해안도로, 용두암, 카멜리아힐, 수목원테마파크, 산방산유람선, 성읍민속마을...","[애월한담해안도로, 용두암, 카멜리아힐, 수목원테마파크, 산방산유람선, 성읍민속마을...","애월한담해안도로,용두암,카멜리아힐,수목원테마파크,산방산유람선,성읍민속마을,아쿠아플라넷"


In [7]:
attraction_db_path = dir_path + '/tourist_attraction_with_rating_sort_by_type.csv'
attraction_db = pd.read_csv(attraction_db_path)
attraction_db.head(2)      

,Unnamed: 0,alltag,contentsid,title,address,roadaddress,tag,introduction,latitude,longitude,...,region1cd.label,region1cd.refId,region2cd.value,region2cd.label,region2cd.refId,repPhoto.descseo,repPhoto.photoid.photoid,repPhoto.photoid.imgpath,repPhoto.photoid.thumbnailpath,인기점수
0,4162,"숲길,걷기/등산,친구,커플,흐림,봄,자연경관,도보여행,도보,숲,단풍,화장실,어려움",CONT_000000000500281,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,제주특별자치도 제주시 조천읍 교래리 산 137-1,"숲길,걷기/등산,친구,커플,흐림,봄,자연경관,도보여행,도보,숲,단풍","제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,...,제주시,region>region1,15.0,조천,region1>15,Saryeoni Forest Path,18168.0,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,1276524.0
1,4074,"섬속의섬,경관/포토,아이,맑음,자연경관,포토스팟,어린이,봄꽃,유채꽃,유네스코,공용주...",CONT_000000000500477,우도(해양도립공원),제주특별자치도 제주시 우도면 연평리 904-1,제주특별자치도 제주시 우도면 삼양고수물길 1,"섬속의섬,경관/포토,아이,맑음,자연경관,포토스팟,어린이,봄꽃,유채꽃,유네스코",소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,...,섬 속의 섬,region>region3,31.0,우도,region3>31,Udo Maritime Park,19203.0,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,1188600.0


In [8]:
attraction_db = attraction_db.loc[:,['contentsid','title']]
attraction_db.head(2)

,contentsid,title
0,CONT_000000000500281,사려니숲길
1,CONT_000000000500477,우도(해양도립공원)


In [9]:
attraction_db['title'].head(30)

0                    사려니숲길
1               우도(해양도립공원)
2     성산일출봉(UNESCO 세계자연유산)
3                      비자림
4                    천지연폭포
5                      만장굴
6                     새별오름
7                   협재해수욕장
8                     섭지코지
9                    카멜리아힐
10     거문오름(UNESCO 세계자연유산)
11                  함덕해수욕장
12                   월정리해변
13                 한라산국립공원
14                     쇠소깍
15           주상절리대(중문대포해안)
16                    정방폭포
17               에코랜드 테마파크
18               애월한담해안산책로
19                   천제연폭포
20                    산굼부리
21                 오설록티뮤지엄
22                   황우지해안
23                   용머리해안
24                  김녕해수욕장
25                   용눈이오름
26                 한라산영실코스
27                    한림공원
28               아쿠아플라넷 제주
29                   메이즈랜드
Name: title, dtype: object

#문자열 내 공백제거

In [10]:
def remove_space(word):
  #print("입력받은 word : ",word)
  return word.replace(" ","")


In [11]:
attraction_db = attraction_db.loc[:,['contentsid','title']]

In [12]:
attraction_db['title'] = attraction_db['title'].apply(remove_space)

In [13]:
attraction_db.iloc[20:30]

,contentsid,title
20,CONT_000000000500283,산굼부리
21,CONT_000000000500457,오설록티뮤지엄
22,CNTS_000000000018383,황우지해안
23,CONT_000000000500471,용머리해안
24,CONT_000000000500083,김녕해수욕장
25,CONT_000000000500466,용눈이오름
26,CNTS_000000000019973,한라산영실코스
27,CONT_000000000500690,한림공원
28,CONT_000000000500565,아쿠아플라넷제주
29,CNTS_000000000001158,메이즈랜드


In [14]:
attraction_db

,contentsid,title
0,CONT_000000000500281,사려니숲길
1,CONT_000000000500477,우도(해양도립공원)
2,CONT_000000000500349,성산일출봉(UNESCO세계자연유산)
3,CONT_000000000500270,비자림
4,CONT_000000000500618,천지연폭포
...,...,...
4265,CNTS_300000000012488,의식주
4266,CNTS_300000000012487,민속
4267,CNTS_300000000012489,삼다도
4268,CNTS_300000000012490,제주의역사


#타이틀 내 괄호 제거

In [15]:
import re
pattern = r'\([^)]*\)'

In [16]:
x='test(괄호)'
text = re.sub(pattern=pattern, repl='',string=x)
text

'test'

In [17]:
attraction_db['title'] = attraction_db['title'].apply(lambda x: re.sub(pattern=pattern,repl='',string=x))
attraction_db['title'].head(30)

0         사려니숲길
1            우도
2         성산일출봉
3           비자림
4         천지연폭포
5           만장굴
6          새별오름
7        협재해수욕장
8          섭지코지
9         카멜리아힐
10         거문오름
11       함덕해수욕장
12        월정리해변
13      한라산국립공원
14          쇠소깍
15        주상절리대
16         정방폭포
17     에코랜드테마파크
18    애월한담해안산책로
19        천제연폭포
20         산굼부리
21      오설록티뮤지엄
22        황우지해안
23        용머리해안
24       김녕해수욕장
25        용눈이오름
26      한라산영실코스
27         한림공원
28     아쿠아플라넷제주
29        메이즈랜드
Name: title, dtype: object

#코스내의 여행지와 attraction_db와의 비교

In [18]:
'사려니숲길' in attraction_db['title'].unique()

True

In [19]:
#course에 있는 destination도 공백 제거 및 괄호 제거
tmp = [['테스트1번 테스트','여기도 (괄호입니다) 괄호      ','   어  어   '],['테스트1번 테스트','여기도 (괄호입니다) 괄호      ','   어  어   ']]
for path in tmp:
  print(type(path))
  path = list(map(remove_space,path)) # 공백 제거
  path = list(map(lambda x: re.sub(pattern=pattern,repl='',string=x),path)) # 괄호 제거
  print(path)
  break

<class 'list'>
['테스트1번테스트', '여기도괄호', '어어']


In [20]:
course['list path(no airport)'].head(7)

0                     [수목원테마파크, 카멜리아힐, 성읍민속마을, 아쿠아플라넷]
1    [애월한담해안도로, 용두암, 카멜리아힐, 수목원테마파크, 산방산유람선, 성읍민속마을...
2           [애월한담해안도로, 수목원테마파크, 카멜리아힐, 성읍민속마을, 아쿠아플라넷]
3                                [용두암, 제주레저승마, 성읍민속마을]
4                [용두암, 산방산유람선, 제주레저승마, 성읍민속마을, 아쿠아플라넷]
5                                [용두암, 제주레저승마, 성읍민속마을]
6                [용두암, 산방산유람선, 제주레저승마, 성읍민속마을, 아쿠아플라넷]
Name: list path(no airport), dtype: object

##관광지만 적용하기(음식점, 숙박 제외)

In [34]:
attraction_db=pd.read_csv(dir_path+'/only_attraction_db.csv')
attraction_db.head(1)

,Unnamed: 0,alltag,contentsid,title,address,roadaddress,tag,introduction,latitude,longitude,...,region1cd.label,region1cd.refId,region2cd.value,region2cd.label,region2cd.refId,repPhoto.descseo,repPhoto.photoid.photoid,repPhoto.photoid.imgpath,repPhoto.photoid.thumbnailpath,인기점수
0,0,"숲길,걷기/등산,친구,커플,흐림,봄,자연경관,도보여행,도보,숲,단풍,화장실,어려움",CONT_000000000500281,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,제주특별자치도 제주시 조천읍 교래리 산 137-1,"숲길,걷기/등산,친구,커플,흐림,봄,자연경관,도보여행,도보,숲,단풍","제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,...,제주시,region>region1,15.0,조천,region1>15,Saryeoni Forest Path,18168.0,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,1276524.0


In [35]:
attraction_db.drop((['Unnamed: 0']),axis=1,inplace=True)

In [36]:
attraction_db

,alltag,contentsid,title,address,roadaddress,tag,introduction,latitude,longitude,postcode,...,region1cd.label,region1cd.refId,region2cd.value,region2cd.label,region2cd.refId,repPhoto.descseo,repPhoto.photoid.photoid,repPhoto.photoid.imgpath,repPhoto.photoid.thumbnailpath,인기점수
0,"숲길,걷기/등산,친구,커플,흐림,봄,자연경관,도보여행,도보,숲,단풍,화장실,어려움",CONT_000000000500281,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,제주특별자치도 제주시 조천읍 교래리 산 137-1,"숲길,걷기/등산,친구,커플,흐림,봄,자연경관,도보여행,도보,숲,단풍","제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.408450,126.639760,NaN,...,제주시,region>region1,15.0,조천,region1>15,Saryeoni Forest Path,1.816800e+04,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,1276524.0
1,"섬속의섬,경관/포토,아이,맑음,자연경관,포토스팟,어린이,봄꽃,유채꽃,유네스코,공용주...",CONT_000000000500477,우도(해양도립공원),제주특별자치도 제주시 우도면 연평리 904-1,제주특별자치도 제주시 우도면 삼양고수물길 1,"섬속의섬,경관/포토,아이,맑음,자연경관,포토스팟,어린이,봄꽃,유채꽃,유네스코",소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.519490,126.951090,63365,...,섬 속의 섬,region>region3,31.0,우도,region3>31,Udo Maritime Park,1.920300e+04,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,1188600.0
2,"일출,오름,경관/포토,부모,자연경관,포토스팟,유네스코,공용주차장,현금결제,카드결제,...",CONT_000000000500349,성산일출봉(UNESCO 세계자연유산),제주특별자치도 서귀포시 성산읍 성산리 1,제주특별자치도 서귀포시 성산읍 일출로 284-12,"일출,오름,경관/포토,부모,자연경관,포토스팟,유네스코","바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제420호, 올레1코스",33.462147,126.936424,63643,...,서귀포시,region>region2,17.0,성산,region2>17,성산일출봉(UNESCO 세계자연유산),2.018052e+12,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,1027366.0
3,"걷기/등산,아이,흐림,여름,힐링,자연경관,어린이,수국,공용주차장,현금결제,카드결제,...",CONT_000000000500270,비자림,제주특별자치도 제주시 구좌읍 평대리 3161-1,제주특별자치도 제주시 구좌읍 비자숲길 55,"걷기/등산,아이,흐림,여름,힐링,자연경관,어린이,수국",천년의 세월이 녹아든 신비로움 가득한 비자림은 500~800년생 비자나무들이 자생하...,33.491318,126.811386,63351,...,제주시,region>region1,16.0,구좌,region1>16,비자림,2.019023e+12,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,916549.0
4,"폭포, 밤, 경관/포토, 부모, 맑음, 여름,공용주차장,현금결제,카드결제,화장실,음...",CONT_000000000500618,천지연폭포,제주특별자치도 서귀포시 서홍동 666-1,제주특별자치도 서귀포시 남성중로 2-15,"폭포,밤,경관/포토,부모,맑음,여름,자연경관,포토스팟,유네스코","하늘과 땅이 만나서 이룬 연못, \r\n· 천연기념물 제163호\r\n· 천연기념물...",33.244717,126.559820,NaN,...,서귀포시,region>region2,21.0,서귀포시내,region2>21,천지연폭포,2.019023e+12,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,645261.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,"1,공용주차장,현금결제,카드결제,화장실,무료 WIFI,카드결제,현금결제,,,어려움,...",CNTS_200000000013506,해빛,제주특별자치도 제주시 조천읍 신촌리 1899-1,제주특별자치도 제주시 조천읍 신촌9길 8,"기념품,상점/상가,체험,힐링","스테인드글래스 공방으로 유리 조명, 썬캐쳐, 각종 소품을 판매하고 원데이 클래스를 ...",33.535320,126.622290,63337,...,제주시,region>region1,15.0,조천,region1>15,해빛,2.019023e+12,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,0.0
1094,"요가,웰니스,공용주차장,카드결제,화장실,무료 WIFI,카드결제,네이버페이, 카카오페...",CNTS_200000000013306,빠우사,제주특별자치도 제주시 조천읍 선흘리 472-5,제주특별자치도 제주시 조천읍 선교로 539,"요가,카페,체험,음식,디저트",그림 명상과 요가를 진행하는 요가카페입니다.,33.457817,126.711838,63341,...,제주시,region>region1,15.0,조천,region1>15,빠우사,2.019023e+12,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,0.0
1095,"힐링,공용주차장,현금결제,카드결제,화장실,무료 WIFI,카드결제,현금결제,,영어,,...",CNTS_200000000013530,요가느림원,제주특별자치도 제주시 노형동 922-6,제주특별자치도 제주시 원노형2길 33,"힐링,요가,체험",18년된 전통요가원,33.483325,126.481684,63084,...,제주시,region>region1,11.0,제주시내,region1>11,요가느림원,2.019023e+12,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,0.0
1096,문화유적지,CNTS_000000000019091,묵리처녀당,제주특별자치도 제주시 추자면 묵리 229-1,제주특별자치도 제주시 추자면 묵리 229-1,"문화유적지,문화관광,역사유적,어트랙션",죽은 처녀의 원혼을 달래주는 추자도 묵리처녀당(할망당),33.943170,126.314260,NaN,...,섬 속의 섬,region>region3,32.0,추차도,region3>32,Mukri Jeonyeodang,4.298000e+03,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,0.0


In [37]:
'수목원테마파크' in attraction_db['title'].unique()

False

In [22]:
similarity('아쿠아플라넷 제주','아쿠아플라넷')

0.8

In [23]:
test_set = set([])
test_set.add((1,2))
print(test_set)

{(1, 2)}


In [24]:
len(course['list path(no airport)'])

6167

In [40]:
'성산일출봉' in attraction_db['title'].unique()

False

In [41]:
test_course_1 = ['성산일출봉(UNESCO 세계자연유산)']
test_course_2 = list(map(remove_space,test_course_1))
test_course_3 = list(map(lambda x: re.sub(pattern=pattern,repl='',string=x),test_course_2)) # 괄호 제거
print(test_course_3)

['성산일출봉']


In [46]:
dest1 = '성산일출봉'
if dest1 in attraction_db['title'].unique():
  print(dest1,'is in db')
else:
  print("없어")
      

없어


In [76]:
attraction_db.loc[attraction_db['title']=='김녕성세기해변',['title']] = '김녕해안도로'
attraction_db.loc[attraction_db['title']=='김녕해안도로']

,alltag,contentsid,title,address,roadaddress,tag,introduction,latitude,longitude,postcode,...,region1cd.refId,region2cd.value,region2cd.label,region2cd.refId,repPhoto.descseo,repPhoto.photoid.photoid,repPhoto.photoid.imgpath,repPhoto.photoid.thumbnailpath,인기점수,title_for_search
554,"드라이브,커플,맑음,여름,공용주차장,화장실,아주 어려움",CNTS_000000000021064,김녕해안도로,제주특별자치도 제주시 구좌읍 김녕리,제주특별자치도 제주시 구좌읍 김녕리,"드라이브,커플,맑음,여름,자연경관",김녕에서 하도까지 바다를 끼고 달리는 길,33.55654,126.7603,NaN,...,region>region1,16.0,구좌,region1>16,김녕해안도로,2.019023e+12,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,8710.0,김녕성세기해변


In [70]:
print(similarity('김녕성세기해변','김녕성세기 해변'))
print(similarity('김녕성세기해변,김녕해안도로','김녕해안도로'))

0.875
0.6


In [77]:
attraction_db.loc[attraction_db['title']=='김녕해수욕장',['title']] = '김녕성세기해변'
attraction_db.loc[attraction_db['title']=='김녕성세기해변']

,alltag,contentsid,title,address,roadaddress,tag,introduction,latitude,longitude,postcode,...,region1cd.refId,region2cd.value,region2cd.label,region2cd.refId,repPhoto.descseo,repPhoto.photoid.photoid,repPhoto.photoid.imgpath,repPhoto.photoid.thumbnailpath,인기점수,title_for_search
24,"해수욕장,액티비티,아이,맑음,여름,공용주차장,화장실,흡연구역,편의점,음료대,유도 및...",CONT_000000000500083,김녕성세기해변,제주특별자치도 제주시 구좌읍 김녕리 4330,제주특별자치도 제주시 구좌읍 구좌해안로 237,"해수욕장,액티비티,아이,맑음,여름,자연경관,체험,레저/체험,해변,물놀이,어린이,수상레저",너럭바위 위에 생긴 김녕해수욕장,33.557926,126.75998,63348,...,region>region1,16.0,구좌,region1>16,Gimnyeong Seonsegi Beach,17248.0,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,185541.0,김녕해수욕장


In [78]:
attraction_db['title_for_search'] = attraction_db['title'].apply(remove_space)
attraction_db['title_for_search'] = attraction_db['title_for_search'].apply(lambda x: re.sub(pattern=pattern,repl='',string=x))

In [79]:
attraction_db['title_for_search']

0        사려니숲길
1           우도
2        성산일출봉
3          비자림
4        천지연폭포
         ...  
1093        해빛
1094       빠우사
1095     요가느림원
1096     묵리처녀당
1097    제주양떼목장
Name: title_for_search, Length: 1098, dtype: object

In [113]:
changed_path = []

In [114]:
cnt = 0
need_to_check_list = set([])
for path in course['list path(no airport)']:
  path = list(map(remove_space,path))
  path = list(map(lambda x: re.sub(pattern=pattern,repl='',string=x),path)) # 괄호 제거
  new_path = "" #변경될 path 저장용
  for dest in path:
    if dest in attraction_db['title_for_search'].unique():
      #print(dest,'is in db')
      new_path += dest + ','
      continue
    else:
      print("now dest is : ",dest)
      max_rate = 0
      save_max = 'not saved'
      token=0
      for attraction in attraction_db['title_for_search']:
        if max_rate < similarity(dest,attraction): # max similarity가 갱신되는 경우
          max_rate = similarity(dest,attraction)
          save_max = attraction

        if max_rate >= 0.80: # 0.8이상인 경우는 그냥 break
     #     print(dest,'->',save_max)
          token=1
          new_path += save_max + ','
          break
      if max_rate <0.66:
        print('abandon value')
        #need_to_check_list.add((dest,save_max)) # 확인 필요한 리스트에 넣자
    #    print(dest,'->',save_max,"이거는 버려지는 값: ",max_rate)
      else:
        if token==0: #중복 저장을 막기위한 토큰값 확인
          new_path += save_max + ','
   #       print(dest,'->',save_max,"일치율 : ",max_rate)
        
  changed_path.append(new_path[:-1])      
  #print("******************************\n")
  # cnt+=1
  # if cnt==300:
  #   break

#print(need_to_check_list)

course['path_only_attraction'] = changed_path
course.to_csv(dir_path+'/course_data_final.csv')

Streaming output truncated to the last 5000 lines.
now dest is :  마레보리조트
abandon value
now dest is :  마레보리조트
abandon value
now dest is :  중문·대포해안주상절리대
abandon value
now dest is :  서귀포매일올레시장
abandon value
now dest is :  마레보리조트
abandon value
now dest is :  동문재래시장
abandon value
now dest is :  JDC면세점
abandon value
now dest is :  흑돈가제주본점
abandon value
now dest is :  칠돈가
abandon value
now dest is :  중문·대포해안주상절리대
abandon value
now dest is :  서귀포네거리식당
abandon value
now dest is :  쌍둥이횟집
abandon value
now dest is :  서귀포매일올레시장
abandon value
now dest is :  옛날옛적
abandon value
now dest is :  서빈백사해수욕장
abandon value
now dest is :  우두봉
now dest is :  명진전복
abandon value
now dest is :  세화해수욕장
now dest is :  김녕해수욕장
now dest is :  동복리해녀촌
abandon value
now dest is :  함덕서우봉해변
abandon value
now dest is :  자매국수
abandon value
now dest is :  제주동문수산시장
abandon value
now dest is :  자매국수
abandon value
now dest is :  올레캐슬펜션
abandon value
now dest is :  함덕서우봉해변
abandon value
now dest is :  올레캐슬펜션
abandon value
now des

In [96]:
course.loc[:,['serial path(no airport)', 'path_only_attraction']]

,serial path(no airport),path_only_attraction
0,"수목원테마파크,카멜리아힐,성읍민속마을,아쿠아플라넷","수목원테마파크,카멜리아힐,성읍민속마을,아쿠아플라넷"
1,"애월한담해안도로,용두암,카멜리아힐,수목원테마파크,산방산유람선,성읍민속마을,아쿠아플라넷","애월한담해안도로,용두암,카멜리아힐,수목원테마파크,산방산유람선,성읍민속마을,아쿠아플라넷"
2,"애월한담해안도로,수목원테마파크,카멜리아힐,성읍민속마을,아쿠아플라넷","애월한담해안도로,수목원테마파크,카멜리아힐,성읍민속마을,아쿠아플라넷"
3,"용두암,제주레저승마,성읍민속마을","용두암,제주레저승마,성읍민속마을"
4,"용두암,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷","용두암,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷"
...,...,...
6162,"자매국수,애월 해안도로,리치 망고,놀맨,협재 해수욕장,천지연 폭포,가름 게스트하우스...","애월해안도로,협재해수욕장,천지연폭포,성산일출봉,김녕해수욕장,만장굴,오설록티뮤지엄,카..."
6163,"흑돈가 제주본점,러브랜드,제주 중앙지하상가,용두암,모나코 호텔,제주 공룡 랜드,리치...","용두암,제주공룡랜드,협재해수욕장,제주유리의성,제주테지움,소인국테마파크,박물관은살아있..."
6164,"망고레이,유리네식당,에코랜드,돈향기,롯데제주리조트아트빌라스,롯데제주리조트아트빌라스,...","에코랜드,쇠소깍,감귤박물관,아쿠아플라넷제주,섭지코지매표소,천지연폭포,이니스프리제주하..."
6165,"오설록 티 뮤지엄,박물관은 살아있다,천지연 폭포,닥종이인형박물관,제주 방주교회,방림...","오설록티뮤지엄,박물관은살아있다,천지연폭포,제주방주교회,방림원,넥슨컴퓨터박물관,국립제..."


In [97]:
course.to_csv(dir_path+'/course_data_final.csv')

##dataframe에 list 넣어지나 테스트

In [90]:
df_test = attraction_db[['title','인기점수']].sample(2)
df_test

,title,인기점수
64,송악산,104079.0
960,연대포구,3274.0


In [91]:
test_list = ['1st string','']
df_test['list test']=test_list
df_test

,title,인기점수,list test
64,송악산,104079.0,1st string
960,연대포구,3274.0,


In [65]:
for idx,val in enumerate(course.loc[80:200,'serial path(no airport)']):
  print(idx,val)

0 용두암,유리의성 제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
1 용두암,유리의성 제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
2 애월한담해안도로,카멜리아힐,서커스월드,수목원테마파크,제주레저승마,성읍민속마을
3 용두암,유리의성 제주,카멜리아힐,제주레저승마,성읍민속마을,우도
4 용두암,유리의성 제주,제주마라도잠수함,제주레저승마,성읍민속마을,섭지코지
5 용두암,유리의성 제주,카멜리아힐,제주레저승마,성읍민속마을,우도
6 용두암,유리의성 제주,카멜리아힐,제주레저승마,성읍민속마을,우도
7 용두암,유리의성 제주,카멜리아힐,제주레저승마,성읍민속마을,우도
8 용두암,유리의성 제주,제주마라도잠수함,제주레저승마,성읍민속마을,섭지코지
9 용두암,유리의성 제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
10 애월한담해안도로,카멜리아힐,서커스월드,수목원테마파크,제주레저승마,성읍민속마을
11 애월한담해안도로,카멜리아힐,서커스월드,수목원테마파크,제주레저승마,성읍민속마을
12 애월한담해안도로,카멜리아힐,서커스월드,수목원테마파크,제주레저승마,성읍민속마을
13 용두암,유리의성 제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
14 용두암,유리의성 제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
15 용두암,한라산,제주레저승마,성읍민속마을,섭지코지
16 용두암,유리의성 제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
17 용두암,유리의성 제주,제주마라도잠수함,제주레저승마,성읍민속마을,섭지코지
18 용두암,유리의성 제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
19 신창해안도로,수월봉,카멜리아힐,마라도,서커스월드,수목원테마파크,제주레저승마,성읍민속마을,아쿠아플라넷
20 용두암,유리의성 제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
21 유리의성 제주,제주마라도잠수함,송악산,제주레저승마,성읍민속마을,섭지코지
22 용두암,유리의성 제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
23 용두암,유리의성 제주,산방산유람선,제주레저

In [66]:
for idx, val in enumerate(changed_path[80:201]):
  print(idx,val)

0 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
1 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
2 애월한담해안도로,카멜리아힐,수목원테마파크,제주레저승마,성읍민속마을
3 용두암,유리의성제주,카멜리아힐,제주레저승마,성읍민속마을,우도
4 용두암,유리의성제주,제주마라도잠수함,제주레저승마,성읍민속마을,섭지코지
5 용두암,유리의성제주,카멜리아힐,제주레저승마,성읍민속마을,우도
6 용두암,유리의성제주,카멜리아힐,제주레저승마,성읍민속마을,우도
7 용두암,유리의성제주,카멜리아힐,제주레저승마,성읍민속마을,우도
8 용두암,유리의성제주,제주마라도잠수함,제주레저승마,성읍민속마을,섭지코지
9 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
10 애월한담해안도로,카멜리아힐,수목원테마파크,제주레저승마,성읍민속마을
11 애월한담해안도로,카멜리아힐,수목원테마파크,제주레저승마,성읍민속마을
12 애월한담해안도로,카멜리아힐,수목원테마파크,제주레저승마,성읍민속마을
13 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
14 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
15 용두암,제주레저승마,성읍민속마을,섭지코지
16 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
17 용두암,유리의성제주,제주마라도잠수함,제주레저승마,성읍민속마을,섭지코지
18 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
19 신창해안도로,수월봉,카멜리아힐,마라도,수목원테마파크,제주레저승마,성읍민속마을,아쿠아플라넷
20 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
21 유리의성제주,제주마라도잠수함,송악산,제주레저승마,성읍민속마을,섭지코지
22 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
23 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,아쿠아플라넷
24 용두암,유리의성제주,산방산유람선,제주레저승마,성읍민속마을,

In [116]:
course['path_only_attraction'].str.get_dummies(sep=',').describe()

,1100고지,1100고지습지,1112도로,5.16도로숲터널,가마오름,가뫼물,가세오름,가파도,가파도마라도정기여객선,가파초등학교마라분교장,...,화순곶자왈생태탐방숲길,화순금모래해수욕장,화조원,환상숲곶자왈공원,황금굴,황우지해안,황우지해안열두굴,후포해변,휴애리자연생활공원,흑돼지거리
count,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,...,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000
mean,0.012486,0.001784,0.001459,0.004865,0.000162,0.000162,0.000162,0.004865,0.000486,0.000486,...,0.002432,0.004702,0.000486,0.014107,0.000162,0.027404,0.008432,0.000324,0.023512,0.006973
std,0.111049,0.042199,0.038177,0.069582,0.012734,0.012734,0.012734,0.069582,0.022052,0.022052,...,0.049262,0.068419,0.022052,0.117943,0.012734,0.163271,0.091445,0.018007,0.151536,0.083217
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
course

In [107]:
course.loc[course['Package_Id']=='334d8b8ff0a258ff',['serial path(no airport)','path_only_attraction']]

,serial path(no airport),path_only_attraction
201,"국수마당,넥슨 컴퓨터 박물관,애월 해안도로,곽지해수욕장,협재해수욕장,협재해녀의집,드...","넥슨컴퓨터박물관,애월해안도로,곽지해수욕장,협재해수욕장,쇠소깍,1100도로,월정리해변..."


In [111]:
attraction_db.loc[attraction_db['title_for_search']=='1100도로']

,alltag,contentsid,title,address,roadaddress,tag,introduction,latitude,longitude,postcode,...,region1cd.refId,region2cd.value,region2cd.label,region2cd.refId,repPhoto.descseo,repPhoto.photoid.photoid,repPhoto.photoid.imgpath,repPhoto.photoid.thumbnailpath,인기점수,title_for_search
